Librerías, paquetes y funciones importadas

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from skimage import transform

In [3]:
def drawBBox(frame, x1, y1, x2, y2, label):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)

def drawPosition(frame, position, position_label):
    cv2.ellipse(frame, (int(position[0]), int(position[1])), (9, 3), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [4]:
# Captura cuatro clics del ratón (parte uno)
def get_points(event, x, y, flags, param):
    points = param["points"]
    img = param["image"].copy()

    if event == cv2.EVENT_LBUTTONDOWN:  # Botón izquierdo
        points.append((x, y))  
        
        # Dibujar en la imagen
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], img)

        # Cuarto puntos, condición de parada
        if len(points) == 4:
            cv2.destroyAllWindows()

# Comprueba si la acción se realiza en el lado derecho del campo
def is_right_side(court_points):
    if len(court_points) < 4:
        print("Error: court_points length must be 4.")
        return False
    return court_points[1][0] < court_points[2][0]


In [5]:
# Puntos de referencia de la zona de 3 segundos en el diagrama
RIGHT_SIDE_POINTS = [(590, 175), (590, 290), (725, 175), (725, 290)]
LEFT_SIDE_POINTS = [(195, 175), (195, 290), (60, 175), (60, 290)]
DIAGRAM_POINTS = [(170, 50), (288, 50), (288, 188), (170, 188)]

# Función para calcular la matriz de homografía a partir de las esquinas detectadas
def calculateHomography(zone_points, is_right_side):
    # Puntos del diagrama para la homografía: [A, B, C, D]
    diagram_points_right_side = [(1509, 436), (1509, 727), (1854, 436), (1854, 727)]
    diagram_points_left_side = [(490, 436), (490, 727), (145, 436), (145, 727)]

    if is_right_side:
        diagram_points = diagram_points_right_side
    else:
        diagram_points = diagram_points_left_side

    # Matriz de homografía
    return transform.estimate_transform('projective', np.array(zone_points), np.array(diagram_points))

def calculateHomographyHalf(zone_points):
    return transform.estimate_transform('projective', np.array(zone_points), np.array(DIAGRAM_POINTS))

# Función para dibujar las detecciones en el diagrama
def drawDetection(homography_matrix, diagram, player_coords):
    diagram_point = homography_matrix(player_coords)
    cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 3, (255, 0, 0), -1)


In [9]:
object_detector = YOLO("../object_detection/runs/detect/bod_v1/weights/best.pt")

video_path = "../../assets/clips/ClipLF1.mp4"
output_path = "../output/ClipLF1_output2.mp4"
diagram = cv2.imread('../../assets/court_diagrams/fiba_white_court_half.png')

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*"avc1")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

court_points = []
diagram_points = []

first_iteration = True

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Crea copias de trabajo
frametmp = frame.copy()
diagramtmp = diagram.copy()

#Vista del campo
cv2.imshow("Vista", frametmp) 
params = {
    "points": court_points,
    "image": frametmp, 
    "wname": "Vista"
}
cv2.setMouseCallback("Vista", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)

if is_right_side(court_points):
    diagram_points = RIGHT_SIDE_POINTS
    print("Lado derecho seleccionado.")
else:
    diagram_points = LEFT_SIDE_POINTS
    print("Lado izquierdo seleccionado.")


# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Coordenadas campo:", court_points)
print("Coordenadas diagrama:", diagram_points)

homography_matrix = transform.estimate_transform(
    'projective',
    np.array(court_points, dtype=np.float32),
    np.array(diagram_points, dtype=np.float32)
)

homography_matrix = calculateHomographyHalf(court_points)

while cap.isOpened():

    if not first_iteration:
        ret, frame = cap.read()
        if not ret:
            break

    frametmp = frame.copy()
    diagramtmp = diagram.copy()

    object_detection_results = object_detector(frame)
    # ball_detection_results = ball_detector(frame)
    # rim_detection_results = rim_detector(frame)

    # Análisis de las detecciones de jugadores
    for result in object_detection_results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if object_detector.names[cls] in ['player'] and conf > 0.5:

            # Calcular la posición como el punto medio del borde inferior de la bbox
            position = (int((x1 + x2) / 2), y2)
            position_label = f"x:{int(position[0])} y:{int(position[1])}"

            label = f"{object_detector.names[cls]} {conf:.2f}"

            # Dibujar la bbox en el frame
            drawBBox(frametmp, x1, y1, x2, y2, label)

            # Dibujar la posición en el frame
            drawPosition(frametmp, position, position_label)

            # Dibujar la detección en el diagrama
            drawDetection(homography_matrix, diagramtmp, position)


    # Mostrar el frame procesado en pantalla
    cv2.imshow('Detecciones', frametmp)
    cv2.imshow('Diagrama', diagramtmp)

    # Esperar por una tecla: espacio para avanzar, 'q' para salir
    key = cv2.waitKey(0) & 0xFF  # Espera indefinidamente hasta que se presione una tecla
    if key == ord('q'):
        break
    elif key == ord(' '):  # Espacio para continuar
        # Escribir el frame procesado en el video de salida
        out.write(frametmp)

    first_iteration = False


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")


Lado izquierdo seleccionado.
Coordenadas campo: [(1263, 553), (1504, 665), (962, 735), (790, 600)]
Coordenadas diagrama: [(195, 175), (195, 290), (60, 175), (60, 290)]

0: 576x1024 10 players, 1 rim, 13.5ms
Speed: 4.3ms preprocess, 13.5ms inference, 1.6ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 10 players, 1 rim, 57.4ms
Speed: 7.2ms preprocess, 57.4ms inference, 5.5ms postprocess per image at shape (1, 3, 576, 1024)
Video procesado guardado en ../output/ClipLF1_output2.mp4
